In [2]:
import torch
import numpy as np
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from scipy.stats import wasserstein_distance
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AffinityPropagation
import matplotlib.pyplot as plt
import random

In [3]:
seed = 1
random.seed(seed)
np.random.seed(seed)

In [4]:
def get_top_values(arr, p):
    # Ensure p is a valid percentage
    if not (0 <= p <= 100):
        raise ValueError("Percentage p must be between 0 and 100.")

    # Calculate the number of elements to retain
    num_elements_to_retain = int(len(arr) * (100 - p) / 100)

    # If num_elements_to_retain is 0, return an empty array
    if num_elements_to_retain == 0:
        return np.array([])

    # Find the indices of the largest num_elements_to_retain elements
    indices_to_retain = np.argpartition(arr, -num_elements_to_retain)[-num_elements_to_retain:]

    # Return only the top (100-p)% of elements
    return arr[indices_to_retain]

In [5]:
def cosine_similarity(A, B):
    A = np.array(A)
    B = np.array(B)

    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        raise ValueError("One of the vectors is zero-vector; cosine similarity is undefined.")

    similarity = dot_product / (norm_A * norm_B)
    return similarity

In [6]:
MODEL_TYPE="alexnet"
DATASET_TYPE=None
NUMBER_OF_CLASSES=10

In [7]:
class Net(nn.Module):
    def init(
        self,
    ):
        super(Net, self).init()

        if MODEL_TYPE == "resnet18":
            self.resnet = models.resnet18(pretrained=False)
            if DATASET_TYPE == "mnist":
                self.resnet18.conv1 = nn.Conv2d(
                    1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
                )
            self.resnet18.fc = nn.Linear(
                self.resnet18.fc.in_features, NUMBER_OF_CLASSES
            )
        elif MODEL_TYPE == "resnet50":
            self.resnet = models.resnet50(pretrained=False)
            self.resnet.fc = nn.Linear(self.resnet.fc.in_features, NUMBER_OF_CLASSES)
        elif MODEL_TYPE == "cnn":
            self.conv1 = nn.Conv2d(1, 6, 5)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(6, 16, 5)
            self.fc1 = nn.Linear(16 * 4 * 4, 120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 10)

        elif MODEL_TYPE == "mobilenet":
            self.mobilenet_v3_large = models.mobilenet_v3_large(pretrained=False)
            self.mobilenet_v3_large.classifier[3] = nn.Linear(self.mobilenet_v3_large.classifier[3].in_features, NUMBER_OF_CLASSES)

        elif MODEL_TYPE == "vgg16":
            self.vgg16 = models.vgg11(pretrained=False)
            self.vgg16.classifier[6] = nn.Linear(4096, NUMBER_OF_CLASSES)

        elif MODEL_TYPE == "alexnet":
            self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  # Output: 32x32
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 16x16
            nn.Conv2d(64, 192, kernel_size=3, padding=1),  # Output: 16x16
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 8x8
            nn.Conv2d(192, 384, kernel_size=3, padding=1),  # Output: 8x8
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),  # Output: 8x8
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Output: 8x8
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 4x4
            )

            self.classifier = nn.Sequential(
                nn.Dropout(p=0.5),
                nn.Linear(256 * 4 * 4, 4096),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.5),
                nn.Linear(4096, 4096),
                nn.ReLU(inplace=True),
                nn.Linear(4096, NUMBER_OF_CLASSES),
            )

    def forward(self, x):
        out = None

        if MODEL_TYPE in ["resnet18", "resnet50"]:
            out = self.resnet(x)
        elif MODEL_TYPE == "cnn":
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(x.size(0), 16 * 4 * 4)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            out = x

        elif MODEL_TYPE == "mobilenet":
            out = self.mobilenet_v3_large(x)

        elif MODEL_TYPE == "vgg16":
            out = self.vgg16(x)

        elif MODEL_TYPE == "alexnet":
            x = self.features(x)
            x = x.view(x.size(0), -1)
            x = self.classifier(x)
            out = x

        return out

In [8]:
def array_prune(array, P):


    array = np.array(array)
    n_elements = array.size

    num_to_zero = int(P*len(arr))
    if num_to_zero == 0:
        num_to_zero = 1

    # Get the indices of the smallest num_to_zero elements
    #smallest_indices = np.argpartition(array, num_to_zero - 1)[:num_to_zero]

    # Create a copy and set the smallest elements to zero
    #modified_array = array.copy()
    #modified_array[smallest_indices] = 0

    #return modified_array

    threshold_value = np.percentile(array, P * 100)

    # Set values below or equal to the threshold to 0
    modified_arr = np.where(array <= threshold_value, 0, array)

    return modified_arr

In [9]:
def remove_batchnorm_params(model):
    state_dict = model.state_dict()

    # Create a new dictionary excluding BatchNorm parameters
    filtered_state_dict = {key: value for key, value in state_dict.items() if "running_mean" not in key and "running_var" not in key}

    #print("Removed BatchNorm parameters:")
    #for key in state_dict.keys():
        #if "running_mean" in key or "running_var" in key:
            #print(f"  - {key}")

    return filtered_state_dict

In [10]:
def remove_batchnorm_params(state_dict):
    # Filter out BatchNorm parameters
    filtered_state_dict = {
        key: value
        for key, value in state_dict.items()
        if "running_mean" not in key and "running_var" not in key and "bn" not in key
    }

    # Print removed BatchNorm parameters for verification
    # print("Removed BatchNorm parameters:")
    # for key in state_dict.keys():
    #     if "running_mean" in key or "running_var" in key or "bn" in key:
    #         print(f"  - {key}")

    return filtered_state_dict
def model_parameters_to_numpy(path):
    # Load the model's state dictionary
    model = torch.load(path, map_location=torch.device('cpu'))
    # model.eval()  # Set the model to evaluation mode
    #print("Model loaded successfully.")
    #model = model.state_dict()

    # Initialize an empty list to store parameters
    params = []

    # Iterate through the model to extract parameters
    model = remove_batchnorm_params(model)
    for name, param in model.items():
        if 'bias' not in name:  # Exclude bias terms
            params.append(param.detach().cpu().numpy())  # Convert to numpy array

    # Return the parameters as a numpy array
    return np.array(params, dtype=object)

In [11]:
def top_to_one_other_zero(arr, P):
    flat_arr = arr.flatten()
    threshold_value = np.percentile(flat_arr, P * 100)

    # Create a new array where values below the threshold are 0 and others are 1
    result = np.where(arr <= threshold_value, 0, 1)

    return result

In [12]:
# Example usage
import numpy as np
arr = np.array([0.5, 2.1, 1.8,0.5, 2.1, -3.3, 0.7, 1.8,0.5, 2.1, -3.3, 0.7, 1.8,0.5, 2.1, -3.3, 0.7, 1.8])
P = .4
result = top_to_one_other_zero(arr, P)
result2 = array_prune(arr, P)
len(arr), result, result2,np.count_nonzero(result) ,np.count_nonzero(result2)

(18,
 array([0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1]),
 array([0. , 2.1, 1.8, 0. , 2.1, 0. , 0.7, 1.8, 0. , 2.1, 0. , 0.7, 1.8,
        0. , 2.1, 0. , 0.7, 1.8]),
 11,
 11)

In [13]:
def group_clients_by_cluster(labels):
    clusters = {}
    for client_id, cluster_id in enumerate(labels):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(client_id)
    return clusters

In [14]:
import numpy as np
from collections import defaultdict
def cluster_to_list(arr):
    cluster_dict = defaultdict(list)
    for index, cluster in enumerate(arr):
        cluster_dict[cluster].append(index)
    clusters = [indices for _, indices in sorted(cluster_dict.items())]

    return clusters

## Load From Lab Google Drive

In [ ]:
!mkdir ./models
!mkdir ./models/alexnet


# 6_epochs_1_fl-rounds
!mkdir ./models/alexnet/6_epochs_1_fl-rounds
!mkdir ./models/alexnet/6_epochs_1_fl-rounds/svhn     # file: https://drive.google.com/drive/folders/1J8jXndhEvbnBXzIMAAXcSknAf105QtCg
!mkdir ./models/alexnet/6_epochs_1_fl-rounds/cifar10  # file: https://drive.google.com/drive/folders/1bN9OMTsZUcrIryKMzNQnVh2gaXaYEFK9

# new_6_epochs_1_fl-rounds
!mkdir ./models/alexnet/new_6_epochs_1_fl-rounds
!mkdir ./models/alexnet/new_6_epochs_1_fl-rounds/svhn    # file
!mkdir ./models/alexnet/new_6_epochs_1_fl-rounds/cifar10 # file

# # 12_epochs_2_fl-rounds
!mkdir ./models/alexnet/12_epochs_2_fl-rounds
!mkdir ./models/alexnet/12_epochs_2_fl-rounds/svhn
!mkdir ./models/alexnet/12_epochs_2_fl-rounds/cifar10

# # 18_epochs_3_fl-rounds
!mkdir ./models/alexnet/18_epochs_3_fl-rounds
!mkdir ./models/alexnet/18_epochs_3_fl-rounds/svhn
!mkdir ./models/alexnet/18_epochs_3_fl-rounds/cifar10

# # 24_epochs_4_fl-rounds
!mkdir ./models/alexnet/24_epochs_4_fl-rounds
!mkdir ./models/alexnet/24_epochs_4_fl-rounds/svhn
!mkdir ./models/alexnet/24_epochs_4_fl-rounds/cifar10

# # 30_epochs_5_fl-rounds
!mkdir ./models/alexnet/30_epochs_5_fl-rounds
!mkdir ./models/alexnet/30_epochs_5_fl-rounds/svhn    # file https://drive.google.com/drive/folders/1kL46fZ6OsPHxxmM7hxNfpRUHxYxt1Hab
!mkdir ./models/alexnet/30_epochs_5_fl-rounds/cifar10 # file https://drive.google.com/drive/folders/1T3gXFegV7Jbvcs79jnoxKBBRy3cH2Rhz

!mkdir ./models/alexnet/new_30_epochs_5_fl-rounds
!mkdir ./models/alexnet/new_30_epochs_5_fl-rounds/svhn    # file https://drive.google.com/drive/folders/1jNxkSgRgThRyKEYRGqY352lwkSoMVaIo?dmr=1&ec=wgc-drive-hero-goto
!mkdir ./models/alexnet/new_30_epochs_5_fl-rounds/cifar10    # file

# 48_epochs_8_fl-rounds
!mkdir ./models/alexnet/48_epochs_8_fl-rounds
!mkdir ./models/alexnet/48_epochs_8_fl-rounds/svhn    # file https://drive.google.com/drive/folders/1ToZM83h1O-T0fwkzA-UElIZVKKW0u_FV
!mkdir ./models/alexnet/48_epochs_8_fl-rounds/cifar10 # file

# 48_epochs_8_fl-rounds
!mkdir ./models/alexnet/new_48_epochs_8_fl-rounds
!mkdir ./models/alexnet/new_48_epochs_8_fl-rounds/svhn    # file https://drive.google.com/drive/folders/1ToZM83h1O-T0fwkzA-UElIZVKKW0u_FV
!mkdir ./models/alexnet/new_48_epochs_8_fl-rounds/cifar10 # file

# 12_epochs_2_pt_fl-rounds
!mkdir ./models/alexnet/12_epochs_2_pt_fl-rounds
!mkdir ./models/alexnet/12_epochs_2_pt_fl-rounds/svhn    # file https://drive.google.com/drive/folders/1ToZM83h1O-T0fwkzA-UElIZVKKW0u_FV
!mkdir ./models/alexnet/12_epochs_2_pt_fl-rounds/cifar10 # file

# 480_epochs_8_pt_fl-rounds
!mkdir ./models/alexnet/480_epochs_8_fl-rounds
!mkdir ./models/alexnet/480_epochs_8_fl-rounds/stl10    # file


# after_aggregation_480_epochs_8_fl-rounds
!mkdir ./models/alexnet/after_aggregation_480_epochs_8_fl-rounds
!mkdir ./models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10    # file

## Downloading 30 epochs 6 FL_round


In [ ]:
# gdown.download('https://drive.google.com/file/d/1CPDtFJjgmv6gCDS_g3aq2h2KRNJ5SwvG/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/svhn/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1mRvYgGOLKjT-LzMiBMFUGY-Rn_ahNR1N/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/svhn/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1-0otn5iE1X9nr_gM9hjJVVyAbWsnMTQH/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/svhn/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1wHHkKtS0lBQekrvpQK3OdabgzMwtqywT/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/svhn/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1PnigwJKYowB-oUe93ezCkExaNOKcEyvE/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/svhn/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1y0NrAI-TK2_TFlOl7ray4--MjscbWP5y/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/svhn/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1vOMdwDPj7IZTuTlWMTCTx-ydkCaa5XxJ/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/svhn/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1urVXQSdGQdU5fPF6vXfMFNFB14l5QAvU/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/svhn/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1EfhIbKV3NYVEzXyJDlFEr3eFnxn9m_I8/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/svhn/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1_mfgGRz6SC9tui07gGWu2FX-HfIPpH7U/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/svhn/node_9.pth', quiet=True,fuzzy=True)

# print("30_epochs_5_fl-rounds svhn models:")
# os.listdir("./models/alexnet/30_epochs_5_fl-rounds/svhn/")

# gdown.download('https://drive.google.com/file/d/14PJEHXEa3CsmB5IZ10Zm2hNXDaM7bwWh/view?usp=drive_link', './models/alexnet/new_30_epochs_5_fl-rounds/svhn/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1dosoXP2MW-KReUgO7_zUzfD0z_Joo0-t/view?usp=drive_link', './models/alexnet/new_30_epochs_5_fl-rounds/svhn/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1D2AlwPw3lEXYFHqePxMZJvZ9HF4IT0mc/view?usp=drive_link', './models/alexnet/new_30_epochs_5_fl-rounds/svhn/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1PP82t74hUL5b8wn98l1n2Tc7x0R25xaj/view?usp=drive_link', './models/alexnet/new_30_epochs_5_fl-rounds/svhn/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1wICtwb1DYbX7EcfYpdpsui9Hcm-0YJPN/view?usp=drive_link', './models/alexnet/new_30_epochs_5_fl-rounds/svhn/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1LGeUpZIdrskaptsYlgJvUmDT72CjeP3r/view?usp=drive_link', './models/alexnet/new_30_epochs_5_fl-rounds/svhn/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1UFBX2nfMNK8wa8smVpfr8L7_XC0VIXgt/view?usp=drive_link', './models/alexnet/new_30_epochs_5_fl-rounds/svhn/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1ei_IgWbsmo2qNHBFx0SviqN_WVrLHJfc/view?usp=drive_link', './models/alexnet/new_30_epochs_5_fl-rounds/svhn/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1RsZ_aZFvK0eUjQsR4aF-IdDbB8gGtoyH/view?usp=drive_link', './models/alexnet/new_30_epochs_5_fl-rounds/svhn/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1krmjPbmpPH6ION6BW-adY3iekLnLCt2y/view?usp=drive_link', './models/alexnet/new_30_epochs_5_fl-rounds/svhn/node_9.pth', quiet=True,fuzzy=True)

# print("new_30_epochs_5_fl-rounds svhn models:")
# os.listdir("./models/alexnet/new_30_epochs_5_fl-rounds/svhn/")


# gdown.download('https://drive.google.com/file/d/1CH-G98zotx2PwCQwCGFvrmpJVJ3ZVtZH/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/cifar10/node_0.pth', quiet=False,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/18aPx6cSZT_XBG4y8VbvKF9hml3A1paE_/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/cifar10/node_1.pth', quiet=False,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1nJ7o6RJeQR6KMc6gsiHvc8RP6v3pUsLA/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/cifar10/node_2.pth', quiet=False,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1m3pAQoSfQZPNZDsh07enqIoyqqsfy4Rl/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/cifar10/node_3.pth', quiet=False,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1OViYIOihwdg41gB3GZo0ef5LPdd0_3cK/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/cifar10/node_4.pth', quiet=False,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1uWvZbklEhTCVimKMg3H9Hf5VwvQwGFl-/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/cifar10/node_5.pth', quiet=False,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1QsAH3C3zjbKivwrhOjAJaqnS-ckyRJim/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/cifar10/node_6.pth', quiet=False,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1UksEbz1cvfo3MSGUHMmSvfQoyDjkPxAT/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/cifar10/node_7.pth', quiet=False,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1uVOmeLcFdF96lJY9jwO6rLtAf3EdRe0C/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/cifar10/node_8.pth', quiet=False,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/12CQNQFJ1Yc_OHEX0AOe-RmvvKz69Dctw/view?usp=drive_link', './models/alexnet/30_epochs_5_fl-rounds/cifar10/node_9.pth', quiet=False,fuzzy=True)

# print("30_epochs_5_fl-rounds cifar10 models:")
# os.listdir("./models/alexnet/30_epochs_5_fl-rounds/cifar10")

## Downloading 6 epochs 1 FL_round


In [ ]:
# gdown.download('https://drive.google.com/file/d/18dBfe0rNKIgZG-RNyxFZXiLxpv6XHwHa/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/svhn/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/15SkSP1JAwIuwPLn508OMDObA1njFOYhl/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/svhn/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1DqT2EagsiGikGQTiMYD3oNJ4YWOhc2Yz/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/svhn/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1NBXok7PhT4FAx6Bnqq9i6WG6XOGbj4SI/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/svhn/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1H2kp08I2BF7X6xW9MqkOgMQvYYBcFf-u/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/svhn/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1QB4QfrGTS-sB4-cHhNrf1oXUDZigrG6_/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/svhn/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1bbT4R0BO3MOPGqpugY0HIKWW8Ffts_xw/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/svhn/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1laORdEojvSvTq04qTxEC_G6_IL5iiiQc/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/svhn/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1H78A0JvYm5I0lvTMHsmkjOFMe4yAKXcH/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/svhn/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1NOwx9TUVUGjr_qZhi1MJwXRv18K0jGsz/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/svhn/node_9.pth', quiet=True,fuzzy=True)


# gdown.download('https://drive.google.com/file/d/1fFZwHBnDRwc4UDU4YDc42vV1DyQ4W2Kf/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/cifar10/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1CPeyEWIRXt_EMvrKVOscYN5xncownbSF/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/cifar10/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1kz998-LUyhmSIAZezEYq3aFodvNF1T5N/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/cifar10/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1EEhXXJcQFE183myQ1v6gU6DgjvjVYqXo/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/cifar10/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1OkQRVv6GxZrzWbHwiTszhIxMH3Bsav_C/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/cifar10/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/15gLhx1BxYHk1wN3SKxRLolcmqD5i32_n/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/cifar10/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1TI6Re2EZrndsvZ1-wdhsp2908NpXz4pJ/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/cifar10/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/146xZfT2_jaSaNjTJH1tHiTQysNzwPNVS/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/cifar10/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1WPVt5ukcHgmD_PtmsoraLlUNs-MOO-23/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/cifar10/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/19Qc1ni4ptK10EKUnp21T5fJQRahHH21C/view?usp=drive_link', './models/alexnet/6_epochs_1_fl-rounds/cifar10/node_9.pth', quiet=True,fuzzy=True)

# print("6_epochs_1_fl-rounds svhn models:")
# os.listdir("./models/alexnet/6_epochs_1_fl-rounds/svhn/")

# print("6_epochs_1_fl-rounds cifar10 models:")
# os.listdir("./models/alexnet/6_epochs_1_fl-rounds/cifar10")


## Downloading 24 epochs 4 FL_round


In [ ]:
# gdown.download('https://drive.google.com/file/d/1Nt-FFvQmUCapwGvv4O9snAsWyF_69kWH/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/svhn/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1MiB4Y-J7knJzmGmeUS_lN1q3BByGDuhY/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/svhn/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1aLTGpDxQL51Fz3spq1LmvcZm8pJ343D1/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/svhn/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1_7Wg-WgTZcAGevR161sPsDUt_-jIdPHg/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/svhn/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1Xdi8hwBYzMkjMn0NGzIcAxbKEZP8hQwc/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/svhn/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1kT9PDdhAJxQUIM189yH3x0hs0IXnwtIA/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/svhn/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1FupJVsVC9ZglcrOArI4oMF987RbX405Y/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/svhn/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1UaCkQ52D1BX88jQRKkM5MA5hHwIXKcB1/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/svhn/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1RIVU2Ya2Pcm1Qt0nYa8w1qx1z9-7iEzS/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/svhn/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1wzrO6JB92bNhf_uGx9NMhv3zWKVpDJ9V/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/svhn/node_9.pth', quiet=True,fuzzy=True)


# gdown.download('https://drive.google.com/file/d/1fFZwHBnDRwc4UDU4YDc42vV1DyQ4W2Kf/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/cifar10/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1CPeyEWIRXt_EMvrKVOscYN5xncownbSF/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/cifar10/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1kz998-LUyhmSIAZezEYq3aFodvNF1T5N/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/cifar10/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1EEhXXJcQFE183myQ1v6gU6DgjvjVYqXo/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/cifar10/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1OkQRVv6GxZrzWbHwiTszhIxMH3Bsav_C/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/cifar10/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/15gLhx1BxYHk1wN3SKxRLolcmqD5i32_n/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/cifar10/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1TI6Re2EZrndsvZ1-wdhsp2908NpXz4pJ/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/cifar10/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/146xZfT2_jaSaNjTJH1tHiTQysNzwPNVS/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/cifar10/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1WPVt5ukcHgmD_PtmsoraLlUNs-MOO-23/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/cifar10/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/19Qc1ni4ptK10EKUnp21T5fJQRahHH21C/view?usp=drive_link', './models/alexnet/24_epochs_4_fl-rounds/cifar10/node_9.pth', quiet=True,fuzzy=True)

# print("24_epochs_4_fl-rounds svhn models:")
# os.listdir("./models/alexnet/24_epochs_4_fl-rounds/svhn/")

# print("24_epochs_4_fl-rounds cifar10 models:")
# os.listdir("./models/alexnet/24_epochs_4_fl-rounds/cifar10")


## Downloading 36 epochs 6 FL_round


In [ ]:
# gdown.download('https://drive.google.com/file/d/13qQGdAe0toTPn2e9F3TDkLzP3JdiYlC0/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/svhn/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/17mh6hrDFvRuzMDflGBfQr9aid5pezL__/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/svhn/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1E-4onrWMuhbVay_CPn3lpHL5bdnzQ9Yz/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/svhn/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1SlO5b3qlBKvUSHNMNGeaq9vQq6O7FbdL/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/svhn/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1nP9BxLAmk89dRaOaZ6tR2J__7AArRkCR/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/svhn/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1fNa2EefhVgp4GRKEI8lXuuzZ3AcXcZV4/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/svhn/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1npBkTr46ZOEf4fGYuWew5pCC6HUAKnON/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/svhn/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1RPwBERMJknwJ0ftLSo7CJmyg9rNyMWhS/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/svhn/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/125szB-3Ye0n01hXJpMrf_rpQ3nbZjYHv/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/svhn/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1rF9x0AhXbq11T1ylAcROpp-gHqjR6D8S/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/svhn/node_9.pth', quiet=True,fuzzy=True)


# gdown.download('https://drive.google.com/file/d/1fFZwHBnDRwc4UDU4YDc42vV1DyQ4W2Kf/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/cifar10/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1CPeyEWIRXt_EMvrKVOscYN5xncownbSF/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/cifar10/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1kz998-LUyhmSIAZezEYq3aFodvNF1T5N/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/cifar10/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1EEhXXJcQFE183myQ1v6gU6DgjvjVYqXo/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/cifar10/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1OkQRVv6GxZrzWbHwiTszhIxMH3Bsav_C/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/cifar10/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/15gLhx1BxYHk1wN3SKxRLolcmqD5i32_n/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/cifar10/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1TI6Re2EZrndsvZ1-wdhsp2908NpXz4pJ/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/cifar10/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/146xZfT2_jaSaNjTJH1tHiTQysNzwPNVS/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/cifar10/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1WPVt5ukcHgmD_PtmsoraLlUNs-MOO-23/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/cifar10/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/19Qc1ni4ptK10EKUnp21T5fJQRahHH21C/view?usp=drive_link', './models/alexnet/36_epochs_6_fl-rounds/cifar10/node_9.pth', quiet=True,fuzzy=True)

# print("36_epochs_6_fl-rounds svhn models:")
# os.listdir("./models/alexnet/36_epochs_6_fl-rounds/svhn/")

# print("36_epochs_6_fl-rounds cifar10 models:")
# os.listdir("./models/alexnet/36_epochs_6_fl-rounds/cifar10")


## Downloading 48 epochs 8 FL_round


In [ ]:
# gdown.download('https://drive.google.com/file/d/1n8RnsqxfMUIPKZAB82E4CKNrY6UPyz-v/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/svhn/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1qJz-PmcABt0qB6g31lxHp5pvffkzirJe/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/svhn/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1lkfaMxCl7s67ZcwJTKqynIVqaTMT1Xx0/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/svhn/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1anmLdav8gZcusx3MuRGgVzuIuBq_YsKr/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/svhn/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/12_s33DqIHt_o4XAGoW3WTTKEl3Clxxg0/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/svhn/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/15HMnPmk_t2KcGey8N7kvJcl3p3-I81zg/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/svhn/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1eoV-T5mkzCLkiaJFefM7AtgZjiWSp6Rk/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/svhn/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1GyWKaCCBISANzpZlVxIoCXefOMmGpFFJ/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/svhn/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1o9I_-Mhg9qQyQ4lkjXMz2oaNbflJwtDJ/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/svhn/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1AbAaCkeosP6MlsFOWsBIc26iX94OVsBx/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/svhn/node_9.pth', quiet=True,fuzzy=True)


# gdown.download('https://drive.google.com/file/d/1Y5riZqti3NNsHtbWHHGgLYQ_FCTnQ7dA/view?usp=drive_link', './models/alexnet/new_48_epochs_8_fl-rounds/svhn/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1iMDxzmy4tPa5vSybZsH6pGYnVElN84UN/view?usp=drive_link', './models/alexnet/new_48_epochs_8_fl-rounds/svhn/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1aMdNlxngw1JjFgvm-FqYrA6QEUodsVB2/view?usp=drive_link', './models/alexnet/new_48_epochs_8_fl-rounds/svhn/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1U0H9dY2HfUSPbM_mIa6cwOaicGeK5jzT/view?usp=drive_link', './models/alexnet/new_48_epochs_8_fl-rounds/svhn/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1wG5cNgcActmwwRUo2gTUGkvTH91hpspv/view?usp=drive_link', './models/alexnet/new_48_epochs_8_fl-rounds/svhn/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1QqVAFu6IuiRwmv-dWxRg2pcS8czqcNfh/view?usp=drive_link', './models/alexnet/new_48_epochs_8_fl-rounds/svhn/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1GDOMRGhbNSVcSphieT8GQevAn57BHPM_/view?usp=drive_link', './models/alexnet/new_48_epochs_8_fl-rounds/svhn/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1eqbESYPM5-Zb83GgcvBICed0zJ_O52sC/view?usp=drive_link', './models/alexnet/new_48_epochs_8_fl-rounds/svhn/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/11dbTRU7Q1AoA1-n2iuHfjC7GWUXK63E8/view?usp=drive_link', './models/alexnet/new_48_epochs_8_fl-rounds/svhn/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1C5a2eir8JqH-QkfWpzrnDV6EX7u_UIwB/view?usp=drive_link', './models/alexnet/new_48_epochs_8_fl-rounds/svhn/node_9.pth', quiet=True,fuzzy=True)


# gdown.download('https://drive.google.com/file/d/1fFZwHBnDRwc4UDU4YDc42vV1DyQ4W2Kf/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/cifar10/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1CPeyEWIRXt_EMvrKVOscYN5xncownbSF/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/cifar10/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1kz998-LUyhmSIAZezEYq3aFodvNF1T5N/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/cifar10/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1EEhXXJcQFE183myQ1v6gU6DgjvjVYqXo/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/cifar10/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1OkQRVv6GxZrzWbHwiTszhIxMH3Bsav_C/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/cifar10/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/15gLhx1BxYHk1wN3SKxRLolcmqD5i32_n/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/cifar10/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1TI6Re2EZrndsvZ1-wdhsp2908NpXz4pJ/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/cifar10/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/146xZfT2_jaSaNjTJH1tHiTQysNzwPNVS/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/cifar10/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1WPVt5ukcHgmD_PtmsoraLlUNs-MOO-23/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/cifar10/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/19Qc1ni4ptK10EKUnp21T5fJQRahHH21C/view?usp=drive_link', './models/alexnet/48_epochs_8_fl-rounds/cifar10/node_9.pth', quiet=True,fuzzy=True)

# print("48_epochs_8_fl-rounds svhn models:")
# os.listdir("./models/alexnet/48_epochs_8_fl-rounds/svhn/")

# print("48_epochs_8_fl-rounds cifar10 models:")
# os.listdir("./models/alexnet/48_epochs_8_fl-rounds/cifar10")


## Downloading 12 epochs 2 FL_round


In [ ]:
# gdown.download('https://drive.google.com/file/d/1LP8XGkLF8e1XyjIKjQkCFXqh6n7Jid2o/view?usp=drive_link', './models/alexnet/12_epochs_2_pt_fl-rounds/svhn/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/13h-kXBTKPX37lAYPaLvHGTOqy_Wcz_ei/view?usp=drive_link', './models/alexnet/12_epochs_2_pt_fl-rounds/svhn/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/19MDBwUnLnwxli4UcDdkoUsOfuPo8LBjL/view?usp=drive_link', './models/alexnet/12_epochs_2_pt_fl-rounds/svhn/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1NzhIdAJHb_cNh-dyAE0NXIp95eBdDTb-/view?usp=drive_link', './models/alexnet/12_epochs_2_pt_fl-rounds/svhn/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1RTq3ziIG8nNpX0T6O2e5Z5ZPgX5zDevn/view?usp=drive_link', './models/alexnet/12_epochs_2_pt_fl-rounds/svhn/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1d6S-QKkW0zocWU9uVofgNaaeH-mMQ22J/view?usp=drive_link', './models/alexnet/12_epochs_2_pt_fl-rounds/svhn/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1Y4mgc9pAkP3uLqAHTqbSlC_E3AD8Qdbw/view?usp=drive_link', './models/alexnet/12_epochs_2_pt_fl-rounds/svhn/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1LKX7G55bL0-SrdgKaeuI3c2tgODd72-I/view?usp=drive_link ', './models/alexnet/12_epochs_2_pt_fl-rounds/svhn/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1ZEuCLYWZM-I6_RKzGBppvynmkzyEJJ2U/view?usp=drive_link', './models/alexnet/12_epochs_2_pt_fl-rounds/svhn/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1n6bz9LZV8bM25ocfCTbybw5UyQJvXBHh/view?usp=drive_link', './models/alexnet/12_epochs_2_pt_fl-rounds/svhn/node_9.pth', quiet=True,fuzzy=True)


## Downloading 6 epochs 1 FL_round


In [ ]:
# gdown.download('https://drive.google.com/file/d/1cf1LKip4N5TM2EuQloWAd-40FV2tFmds/view?usp=drive_link', './models/alexnet/new_6_epochs_1_fl-rounds/svhn/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1f1SjBlUPhGJmxN-WDjiMRZUorvfqI-OT/view?usp=drive_link', './models/alexnet/new_6_epochs_1_fl-rounds/svhn/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1DjH8vuYv78ZH6eduYHucf9K0GlgV69jJ/view?usp=drive_link', './models/alexnet/new_6_epochs_1_fl-rounds/svhn/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1_uadFY7FzNWdtH-zuQqm8URFxDbTaFic/view?usp=drive_link', './models/alexnet/new_6_epochs_1_fl-rounds/svhn/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1BPegtFfrqnJ-4FYF4GfAQDkSz_gYg9fb/view?usp=drive_link', './models/alexnet/new_6_epochs_1_fl-rounds/svhn/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1MrzRie5h0Nk7coH1P5b2TP1kqf7H8QwD/view?usp=drive_link', './models/alexnet/new_6_epochs_1_fl-rounds/svhn/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1EML-gcseizFOTv3yMIddUng_QrzWAMwi/view?usp=drive_link', './models/alexnet/new_6_epochs_1_fl-rounds/svhn/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/14B57dTMH1u-EmWyjUd6K3yKhpdvtKy5L/view?usp=drive_link', './models/alexnet/new_6_epochs_1_fl-rounds/svhn/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1wts-ooHbD8MdEmVTj3wFwFiWWMGsL8YS/view?usp=drive_link', './models/alexnet/new_6_epochs_1_fl-rounds/svhn/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1Q58XMB6n8k1Fijz1rcctzz_YTy8hvrPh/view?usp=drive_link', './models/alexnet/new_6_epochs_1_fl-rounds/svhn/node_9.pth', quiet=True,fuzzy=True)


## Downloading 480 epochs 6 FL_round


In [ ]:
# gdown.download('https://drive.google.com/file/d/1FwYkt-RbRWsv3W4sbrmIIl0eDk4jieCu/view?usp=drive_link', './models/alexnet/480_epochs_8_fl-rounds/stl10/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1DU6F4EdbJrQB6l_Y7mOqlCK7G4QmTjVT/view?usp=drive_link', './models/alexnet/480_epochs_8_fl-rounds/stl10/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1N705njgTMi2JBdw2OxUX66H7CDfBEFWS/view?usp=drive_link', './models/alexnet/480_epochs_8_fl-rounds/stl10/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1dXslrCGKANfkjfPkrkFR5wlkFG4DRYWk/view?usp=drive_link', './models/alexnet/480_epochs_8_fl-rounds/stl10/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/16ALKh4p_sav6Krng9pinYPDhaWW974N4/view?usp=drive_link', './models/alexnet/480_epochs_8_fl-rounds/stl10/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1hgs5e8rATTnxVfXqUWHr6H_LrKrgJ70B/view?usp=drive_link', './models/alexnet/480_epochs_8_fl-rounds/stl10/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1EArdck5iWL0PldbtW48VXQ8aUjL5AeJk/view?usp=drive_link', './models/alexnet/480_epochs_8_fl-rounds/stl10/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1uyv-n4qrCmqXxpPxs_Pb9YQB0sJ3hyNz/view?usp=drive_link', './models/alexnet/480_epochs_8_fl-rounds/stl10/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/15nGge5zfjZWa3TigCvI2aacr9rE-XvII/view?usp=drive_link', './models/alexnet/480_epochs_8_fl-rounds/stl10/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1CeONwEJfz-hoPTs6t_9SPWFEmHDfGICl/view?usp=drive_link', './models/alexnet/480_epochs_8_fl-rounds/stl10/node_9.pth', quiet=True,fuzzy=True)

## Downloading 480 epochs 6 FL_round After aggregation


In [ ]:
# gdown.download('https://drive.google.com/file/d/1sz78CSsPe2Qg0fYbCEhGPwyNq4ftb1Ig/view?usp=drive_link', './models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10/node_0.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1ctTu9SrnX4rNaTS7yTkwqVWYd1L1-V68/view?usp=drive_link', './models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10/node_1.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/17XrBq7pK1sOuM1yFKB_FNGWF9OdFkBqT/view?usp=drive_link', './models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10/node_2.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1dUCimIynL-PcQxFa36VaB7qsEt_7ql3E/view?usp=drive_link', './models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10/node_3.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1JdVvGdjJ6bEkEQ7MIVmPT72NFlVAUoRm/view?usp=drive_link', './models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10/node_4.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1KO6Vu3pQmOcZnKOz457CJVQbryDO7WTx/view?usp=drive_link', './models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10/node_5.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1kBss9ETU0IDRNLI1fzcxjuBmxJCZxldU/view?usp=drive_link', './models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10/node_6.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1-OFFlAXAIb0joZg2slmFc0sje-lI2hrx/view?usp=drive_link', './models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10/node_7.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1XKpNfQ4T4E0zVclozeyu9U4hwKW1LwNk/view?usp=drive_link', './models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10/node_8.pth', quiet=True,fuzzy=True)
# gdown.download('https://drive.google.com/file/d/1cXFW6j8qIyGs54CbnDNAvGNqrN09yKQo/view?usp=drive_link', './models/alexnet/after_aggregation_480_epochs_8_fl-rounds/stl10/node_9.pth', quiet=True,fuzzy=True)

## Cosine

In [15]:
N=10
params=[]
NUMBER_OF_EPOCHS = 480 # values are: 6, 12, 18, 24, 30, 36, 42, 48, 480
DATASET = "svhn"  # values are: cifar10 or svhn

sub_path = None
pretrained = True
new = True

# if NUMBER_OF_EPOCHS == 6:
#   if pretrained:
#     sub_path = f"new_6_epochs_1_fl-rounds/{DATASET}"
#   else:
#     sub_path = f"6_epochs_1_fl-rounds/{DATASET}"
# elif NUMBER_OF_EPOCHS == 12:
#   if pretrained:
#     sub_path = f"12_epochs_2_pt_fl-rounds/{DATASET}"
#   else:
#     sub_path = f"12_epochs_2_fl-rounds/{DATASET}"
# elif NUMBER_OF_EPOCHS == 18:
#   sub_path = f"18_epochs_3_fl-rounds/{DATASET}"
# elif NUMBER_OF_EPOCHS == 24:
#   sub_path = f"24_epochs_4_fl-rounds/{DATASET}"
# elif NUMBER_OF_EPOCHS == 30:
#   if new:
#     sub_path = f"new_30_epochs_5_fl-rounds/{DATASET}"
#   else:
#     sub_path = f"30_epochs_5_fl-rounds/{DATASET}"
# elif NUMBER_OF_EPOCHS == 36:
#   sub_path = f"36_epochs_6_fl-rounds/{DATASET}"
# elif NUMBER_OF_EPOCHS == 42:
#   sub_path = f"42_epochs_7_fl-rounds/{DATASET}"
# elif NUMBER_OF_EPOCHS == 48:
#   if new:
#     sub_path = f"new_48_epochs_8_fl-rounds/{DATASET}"
#   else:
#     sub_path = f"48_epochs_8_fl-rounds/{DATASET}"
# elif NUMBER_OF_EPOCHS == 480:
#   sub_path = f"480_epochs_8_fl-rounds/stl10"
# else:
#   print("values not supported")

# sub_path = "after_aggregation_480_epochs_8_fl-rounds/stl10"

for i in range(3):
    path = f"./models/alex-30/node_{i}.pth"
    if not os.path.exists(path):
        print(f"File {path} does not exist. Skipping...")
        continue
    try:
        node = model_parameters_to_numpy(path)
        param = np.concatenate([p.ravel() for p in node])
        params.append(param)
        print(f"Successfully loaded: {path}")
    except Exception as e:
        print(f"Error loading {path}: {e}")

/tmp/ipykernel_124531/3809231039.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(path, map_location=torch.device('cpu'))


Successfully loaded: ./models/alex-30/node_0.pth
Successfully loaded: ./models/alex-30/node_1.pth
Successfully loaded: ./models/alex-30/node_2.pth


In [20]:
x = [
[4948, 0,5293, 0, 0,0,0,0,0,0,],
[0,3466, 0, 0, 0,0,0,0,0, 2330,],
[0,3465,5292, 0, 0,0,0,0,0,0,],
[0, 0, 0,4249,3729,0,0,0,0,0,],
[0,3465, 0, 0,3729,0,0,0,0,0,],
[0, 0, 0, 0, 0, 6882,0, 1865,0,0,],
[0, 0, 0, 0, 0,0, 2864, 1865,0,0,],
[0, 0, 0, 0, 0,0, 2863, 1865,0,0,],
[0, 0, 0,4248, 0,0,0,0, 5045,0,],
[0,3465, 0, 0, 0,0,0,0,0, 2329,],]

In [16]:
params[0], params[1], params[2]

(array([ 0.00579114,  0.00879585,  0.0016489 , ..., -0.0073623 ,
        -0.00880101, -0.00668311], dtype=float32),
 array([ 0.00593197,  0.00881935,  0.00101769, ..., -0.00430206,
        -0.00672888, -0.00352799], dtype=float32),
 array([ 0.00573266,  0.00860115,  0.00092842, ..., -0.00673541,
        -0.00833055, -0.00609909], dtype=float32))

In [ ]:
N = len(params)
print("Number of parameters (N):", N)
Cosine_similarity = np.zeros((N, N))
for i in range(N):
    for j in range(i + 1, N):
        Cosine_similarity[i][j] = cosine_similarity(params[i], params[j])

In [ ]:
Cosine_distance = 1 - Cosine_similarity
ap = AffinityPropagation(affinity='precomputed', random_state=0).fit(Cosine_distance)
cosine_labels = ap.labels_

clusters = group_clients_by_cluster(cosine_labels)

print("Clients clustering based on their dataset:", clusters)
print("Cluster labels for each parameter:", cosine_labels)

**Coordinate-Based**

In [ ]:
coordinates=[]
for i in range(N):
  tops=top_to_one_other_zero(params[i], P)
  coordinates.append(tops)
ID_similarity=np.zeros((N,N))
for i in range(N):
  for j in range(i+1,N):
    ID_similarity[i][j]=cosine_similarity(coordinates[i],coordinates[j])

ID_distance = -ID_similarity
ap = AffinityPropagation(affinity="precomputed", random_state=0).fit(ID_distance)
ID_labels = ap.labels_
ID_similarity_show = np.round(ID_similarity, 2)


In [ ]:
Cosine_similarity_show=np.round(Cosine_similarity, 2)
print(cosine_labels)
Cosine_similarity

In [ ]:
print(ID_labels)
ID_similarity

In [ ]:
matrix1=Cosine_similarity.flatten()
matrix2=ID_similarity.flatten()
from scipy.stats import pearsonr, spearmanr

corr_pearson, _ = pearsonr(matrix1, matrix2)
corr_spearman, _ = spearmanr(matrix1, matrix2)
mae = np.mean(np.abs(matrix1 - matrix2))
diff=matrix1- matrix2
min_,max_,avg_=np.min(diff),np.max(diff),np.average(diff)
corr_pearson, corr_spearman, mae, min_,max_,avg_

In [ ]:
matrix2

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

def find_regression_coefficients(x, y):
    x = np.array(x).reshape(-1, 1)
    y = np.array(y)

    # Create and fit the regression model
    model = LinearRegression()
    model.fit(x, y)

    return model.coef_[0], model.intercept_
slope, intercept = find_regression_coefficients(matrix1, matrix2)
cofficient=f"cosine(x,y)={round(slope,3)} ID_sim(x,y) {round(intercept,3)}"
cofficient

In [ ]:
# Filter non-zero values for both matrices
non_zero_indices_x = np.nonzero(matrix1)
non_zero_indices_y = np.nonzero(matrix2)

# Get non-zero values
x_non_zero = matrix1[non_zero_indices_x]
y_non_zero = matrix2[non_zero_indices_y]

# Create scatter plot for non-zero values
plt.figure(figsize=(8, 6))

# Plot X values (Cosine Similarity) in blue
plt.scatter(range(len(x_non_zero)), x_non_zero, color='blue', alpha=0.7, label='Cosine Similarity (X)')

# Plot Y values (Approximation Similarity) in red
plt.scatter(range(len(y_non_zero)), y_non_zero, color='red', alpha=0.7, label='Approximation Similarity (Y)')

# Add labels and title
plt.xlabel('Data Index', fontsize=12)
plt.ylabel('Similarity Values', fontsize=12)
plt.title(f"Pearson Correlation: {round(corr_pearson,3)} Spearman Correlation: {round(corr_spearman,3)} "+cofficient, fontsize=14)
plt.legend()
plt.grid(alpha=0.4)

plt.show()





## **Clustering and Silouhette**

In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.cluster import AffinityPropagation
from sklearn.metrics import pairwise_distances
data = np.array([
    [6000, 0, 3000, 0, 0, 0, 0, 0, 0, 0],
    [0, 1500, 0, 0, 0, 0, 0, 0, 0, 3000],
    [0, 1500, 3000, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 3000, 3000, 0, 0, 0, 0, 0],
    [0, 1500, 0, 0, 3000, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 6000, 0, 2000, 0, 0],
    [0, 0, 0, 0, 0, 0, 3000, 2000, 0, 0],
    [0, 0, 0, 0, 0, 0, 3000, 2000, 0, 0],     # Added rows assuming the pattern.
    [0, 0, 0, 3000, 0, 0, 0, 0, 6000, 0],
    [0, 1500, 0, 0, 0, 0, 0, 0, 0, 3000]
])

# Calculate cosine similarity
cosine_dist_opt = pairwise_distances(data, metric='cosine')
cosine_similarities_opt = -cosine_dist_opt  # Convert distances to negative similarities for AP

# Determine the preference based on the cosine similarity matrix
preference = np.median(cosine_similarities_opt)

# Affinity Propagation with cosine similarity
clustering = AffinityPropagation(affinity='precomputed', preference=preference, random_state=42).fit(cosine_similarities_opt)

cluster_labels = clustering.labels_
# Calculate the silhouette score


silhouette_avg_opt = silhouette_score(data[:9], cluster_labels, metric='cosine')
silhouette_avg_cosine = silhouette_score(data, cosine_labels, metric='cosine')
silhouette_avg_method = silhouette_score(data, ID_labels, metric='cosine')


# Organizing the final clusters as [[], [], ...]
num_clusters = len(np.unique(cluster_labels))
clusters = [[] for _ in range(num_clusters)]
for index, label in enumerate(cluster_labels):
    clusters[label].append(index)

# Output the clusters and silhouette score
print(f"Silhouette Score with[{clusters}]: {silhouette_avg_opt}")
print(f"Silhouette Score_cosine with [{cluster_to_list(cosine_labels)}]: {silhouette_avg_cosine}")
print(f"Silhouette Score_ID method with [{cluster_to_list(ID_labels)}]: {silhouette_avg_method}")


## Others

In [ ]:
similarity_vector=[]
for p in range (0,100):
    similarity_vector.append(cosine_similarity(params[1], array_prune(params[1], p/100)))

In [ ]:
similarity_vector_ID=[]
for p in range (1,100):
    similarity_vector_ID.append(cosine_similarity(params[1], top_to_one_other_zero(params[1], p/100)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
values = np.random.rand(100)
plt.figure(figsize=(10, 5))  # Optional: Adjust the figure size
plt.plot(similarity_vector, marker='o', linestyle='-', label='similarity of prun')
plt.plot(similarity_vector_ID, marker='o', linestyle='-', label='similarity of ID')
plt.title('Plot of 100 Values')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

## Euclidean

In [ ]:
Eu_similarity=np.zeros((N,N))
for i in range(N):
    for j in range(i + 1, N):
        # Calculate Euclidean distance
        Eu_similarity[i][j] =Eu_similarity[j][i] = np.linalg.norm(params[i] - params[j])

Eu_distance = -Eu_similarity
ap = AffinityPropagation(affinity="precomputed").fit(Eu_distance)
labels = ap.labels_

print("Cluster labels for each prune:", labels)

## Wasserstein (!!!Takes too long!!!!)

In [ ]:
W_similarity=np.zeros((N,N))
for i in range(N):
  for j in range(i + 1, N):
    W_similarity[i][j] =W_similarity[j][i] = wasserstein_distance(params[i], params[j])

W_distance=W_similarity
ap = AffinityPropagation(affinity="precomputed").fit(W_distance)
labels = ap.labels_

print("Cluster labels for each prune:", labels)

In [ ]:
a=params[0]
b=params[1]
c=params[2]

x=np.linalg.norm(b)
x

In [ ]:
y=np.linalg.norm(c)
y

In [ ]:
z=np.dot(b,c)
z

In [ ]:
print(z,x,y,x*y,z/(x*y))

In [ ]:
z/x*y

In [ ]:
# b=params[0]
# #a_=prun_0
# #b_=prun_1
# eps_a=np.linalg.norm(a)-np.linalg.norm(a_)
# eps_b=np.linalg.norm(b)-np.linalg.norm(b_)
# print("epsilons:", eps_a, eps_b)
# if eps_a<=np.linalg.norm(a-a_):
#   print("satisfy")

# if eps_b<=np.linalg.norm(b - b_):
#   print("satisfy")
# delta=cosine_similarity(a,b)-cosine_similarity(a_,b_)
# print(delta)

# check_delta=eps_a/np.linalg.norm(a)+eps_b/np.linalg.norm(b)+(eps_a*eps_b)/np.linalg.norm(a)*np.linalg.norm(b)
# print(check_delta)

